In [3]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [4]:
%load_ext autoreload
%autoreload 2
import logging
import multiprocessing
import os
import shutil
import sys
import time
import types

import numpy as np
import pandas as pd

from linajea.config import TrackingConfig
import linajea.evaluation

import linajea.config
import linajea.process_blockwise
import linajea.utils
from blockwise.candidate_graph_blockwise import extract_edges_blockwise

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import zarr
z = zarr.open ( 'anno_alice_T2030_tiny.zarr' , 'r' )

fragments = z['Fragments'][:]
raw = z['Raw'][:]
#track = z['gt_trackimage']
print(z.tree(level=2))

/
 ├── .zarr
 ├── Fragment_stats
 │   ├── Position
 │   ├── Volume
 │   └── id
 ├── Fragments (3, 15, 30, 30) uint64
 ├── Merge_tree
 │   ├── Merge
 │   └── Scoring
 ├── Raw (4, 3, 15, 30, 30) float64
 └── gt_trackimage (3, 15, 30, 30) uint64


In [2]:

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory:", cwd)

Current working directory: /groups/funke/home/xuz2/fatepredict_experiments/example


In [15]:
config_file = 'config.toml'
config = TrackingConfig.from_file(config_file)

In [16]:
extract_edges_blockwise(config)

cabdidategraph_extract_edges ▶:   0%|          | 0/4 [00:00<?, ?blocks/s]


Execution Summary
-----------------

  Task cabdidategraph_extract_edges:

    num blocks : 4
    completed ✔: 4 (skipped 0)
    failed    ✗: 0
    orphaned  ∅: 0

    all blocks processed successfully


In [17]:
data = config.inference_data.data_source
graph_provider = linajea.utils.CandidateDatabase(data.db_name,
                                                config.general.db_host,
                                                mode='r+')

In [49]:
import daisy
extract_roi = daisy.Roi(offset=data.roi.offset,
                            shape=data.roi.shape)

In [66]:
block_write_roi = daisy.Roi(
        (0,)*4,
        daisy.Coordinate(config.extract.block_size))

    # set thresgold
max_edge_move_th = max(config.extract.edge_move_threshold.values())
pos_context = daisy.Coordinate((0,) + (max_edge_move_th,)*3)
neg_context = daisy.Coordinate((1,) + (max_edge_move_th,)*3)


input_roi = extract_roi.grow(neg_context, pos_context)
block_read_roi = block_write_roi

In [67]:
block_read_roi

[0:2, 0:4, 0:4, 0:4] (2, 4, 4, 4)